In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

This is second version of this project. My main purposes are to try many different ML algorithms, approaches, and ways of dealing with problems. I am begginer in ML, so if you have cought any errors or have a suggestion, feel free to write it in a comment :)

In [ ]:
train = pd.read_csv('/kaggle/input/predict-volcanic-eruptions-ingv-oe/train.csv')

In [ ]:
train.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data_dir = '/kaggle/input/predict-volcanic-eruptions-ingv-oe/train/'

In [ ]:
# function to read data

def combine_csv():
    def get_data(index):
        data = pd.read_csv(data_dir + str(train.segment_id.iloc[index]) + ".csv")
        data['time_to_eruption'] = train['time_to_eruption'].iloc[index]
        for feat in data.drop('time_to_eruption',1).columns:
            data[feat] = data[feat].mean()
        data = data.sample(1)
        return data
    
    def combine():
        data = pd.DataFrame()
        for i in range(train.shape[0]):
            df = get_data(i)
            data=pd.concat([df,data])
        return data
    
    return combine()

data = combine_csv()

In [ ]:
# filling null vals with median

for col in data.columns:
    median = data[col].median()
    data[col].fillna(median, inplace=True)
    
data.isnull().sum()

In [ ]:
corr_mx = data.corr()
sns.heatmap(corr_mx)

In [ ]:
# Modeling

from sklearn.model_selection import train_test_split

X = data.drop(['time_to_eruption'], inplace=False, axis=1)
y = data['time_to_eruption']


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape
X_train = np.array(X_train)
y_train = np.array(y_train)

X_train=X_train.reshape(-1,10,1)

X_train.shape

In [ ]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA, KernelPCA
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans, DBSCAN, MiniBatchKMeans
from sklearn.ensemble import AdaBoostRegressor, VotingRegressor, BaggingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras.optimizers import SGD

#model = keras.models.Sequential()
#model.add(keras.layers.Dense(2835, activation = "relu"))
#model.add(keras.layers.Dense(2835, activation = "relu"))
#model.add(keras.layers.Dense(2835, activation = "relu"))
#model.add(keras.layers.Dense(2835, activation = "relu"))
#model.add(keras.layers.Dense(2835, activation = "relu"))
#model.add(keras.layers.Dense(2835, activation = "relu"))
#model.add(keras.layers.Dense(2835, activation = "relu"))
#model.add(keras.layers.Dense(2835, activation = "relu"))
#model.add(keras.layers.Dense(2835, activation = "relu"))
#model.add(keras.layers.Dense(1, activation = "linear"))

#sgd = SGD(learning_rate = 0.001)
#model.compile(optimizer= 'adam' ,loss= 'mean_absolute_error',metrics=['mean_absolute_error'])
#model.fit(X_train, y_train,epochs=5, batch_size=250, validation_data=(X_val, y_val))


model = keras.models.Sequential([
keras.layers.Conv1D(128, 2, activation = "relu", padding = 'valid'),
keras.layers.MaxPool1D(2),
keras.layers.Conv1D(128, 2, activation = "relu", padding = 'valid'),
keras.layers.Conv1D(128, 2, activation = "relu", padding = 'valid'),
keras.layers.MaxPool1D(2),
keras.layers.Flatten(),
keras.layers.Dense(128, activation = "relu"),
keras.layers.Dense(128, activation = "relu"),
keras.layers.Dense(128, activation = "relu"),
keras.layers.Dense(128, activation = "relu"),
keras.layers.Dense(128, activation = "relu"),
keras.layers.Dense(128, activation = "relu"),
keras.layers.Dense(64, activation = "relu"),
keras.layers.Dense(1)
])
sgd = SGD(learning_rate = 0.001)
model.compile(optimizer= 'adam' ,loss= 'mean_absolute_error',metrics=['mean_absolute_error'])
model.fit(X_train, y_train, epochs=30, batch_size=5)
model.summary()

First thing i want to see is how this data reacts to reductions in dimensionality, since this is a topic i've recently learned, and i can't wait to see what i can do with it! ( I still don't understand it fully, so for this version my explorations are left quite shallow )

Not much difrence compared to PCA. Later in this notebook I'll see how PCA/KPCA scales with my other methods.


Also recently, I've read about data preprocessing with clustering algorithms, and I am determined to find something usefull here!

Better that just PCA, but worse that just KMeans, which means that dimensionality reduction in case of this model in counterproductive

In [ ]:
# print(f'Decision Tree: {kmeans(dtr)}\nSupport Vector Machines: {kmeans(SVR())}\nLinear Regression: {kmeans(LinearRegression())}')

In [ ]:
# Best params:
# Decision Tree: (173512024577686.5, {'kmeans__n_clusters': 450})
# Support Vector Machines: (182206672899156.47, {'kmeans__n_clusters': 1500})
# Linear Regression: (6.241083175756122e+16, {'kmeans__n_clusters': 150})

In [ ]:
# I noticed that mse seems to be minimized at around n_clusters=1000 ( Lets forget linear regression )

In [ ]:
# Best params:
# Decision Tree: (170436558910960.75, {'minikmeans__batch_size': 500, 'minikmeans__n_clusters': 100})
# Support Vector Machines: (182206653608092.66, {'minikmeans__batch_size': 700, 'minikmeans__n_clusters': 1})
# Linear Regression: (164458165654294.5, {'minikmeans__batch_size': 200, 'minikmeans__n_clusters': 100})
# Linear regression seems to improve the most, later might tweak n_clusters_param

It seems that MiniBatchKMeans works better that regular KMeans

**Ensemble learning**

In [ ]:
sample = pd.read_csv('/kaggle/input/predict-volcanic-eruptions-ingv-oe/sample_submission.csv')


In [ ]:
test = pd.read_csv("../input/predict-volcanic-eruptions-ingv-oe/sample_submission.csv")

'''Function to get test data'''
def get_csv_test(index):
    
    test_data = pd.read_csv(test_dir + str(test.segment_id.iloc[index]) + ".csv")
    
    for feat in test_data.columns:
        test_data[feat] = test_data[feat].mean()
    
    test_data = test_data.sample()
    
    return(test_data)

In [ ]:
data_test = pd.DataFrame()
test_dir = '/kaggle/input/predict-volcanic-eruptions-ingv-oe/test/'

for index in range(test.shape[0]):
    data_test = pd.concat([get_csv_test(index), data_test])

In [ ]:
data_test.shape


In [ ]:
for i in data_test:
    data_test[i] = data_test[i].replace(np.nan, data_test[i].mean())
data_test.isnull().sum()

In [ ]:
data_test.head()

In [ ]:
X = ['sensor_1','sensor_2','sensor_3','sensor_4','sensor_5','sensor_6','sensor_7','sensor_8','sensor_9','sensor_10']
test_X = data_test[X]
test_X = np.array(test_X)
test_X=test_X.reshape(-1,10,1)
predicted_time = model.predict(test_X)
print(predicted_time)

In [ ]:
test['time_to_eruption'] = predicted_time
sub = test[['segment_id', 'time_to_eruption']]

In [ ]:
sub.head()
sub.to_csv('submission.csv',index=False)